In [62]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import quantstats as qs
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns


DATA_DIR = (os.getcwd() + "/data/")
PREDICTION_DIR = (os.getcwd() + "/predictions/")
start_date = '2014-01-01'
end_date = "2019-12-31"
# Get fundamental data for each stock in the ticker and append to the dataframe




def get_all_symbols():
    return [v.strip('.csv') for v in os.listdir(DATA_DIR)]

def get_current_predictions():
    return [v.strip('_predictions.csv') for v in os.listdir(PREDICTION_DIR)]


tickers = get_all_symbols()
predicted_tickers = get_current_predictions()

sample_index = pd.read_csv(DATA_DIR + 'AAPL.csv', index_col='Date', parse_dates=True)
mask = (sample_index.index >= start_date) & (sample_index.index <= end_date)
sample_index = sample_index.loc[mask]


In [63]:
sample_index

,Open,High,Low,Close,Adj Close,Volume,Returns,Short Term Reversal,Stock Momentum,Long Term Reversal,...,volatility_60,annual_volatility,RSI(2),RSI(7),RSI(14),CCI(30),CCI(50),CCI(100),BBWidth,Williams
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,19.845715,19.893929,19.715000,19.754642,17.341541,234684800,0.985936,-0.025614,-0.025614,-0.023291,...,0.093592,0.287729,26.351929,44.149632,50.996184,13.043634,51.765894,103.220487,0.049720,-56.680804
2014-01-03,19.745001,19.775000,19.301071,19.320715,16.960621,392467600,0.978034,-0.040645,-0.040645,-0.042513,...,0.095361,0.288308,9.511113,33.094339,43.883530,-39.262956,16.963800,78.240752,0.056917,-93.409847
2014-01-06,19.194643,19.528570,19.057142,19.426071,17.053106,412610800,1.005453,-0.028852,-0.028852,-0.042208,...,0.094918,0.286960,30.941907,37.525673,45.858023,-69.532282,2.190718,67.484527,0.058951,-73.014620
2014-01-07,19.440001,19.498571,19.211430,19.287144,16.931154,317209200,0.992849,-0.026113,-0.026113,-0.035677,...,0.095235,0.286990,19.045879,34.055627,43.675706,-82.255350,0.736180,65.854396,0.064762,-83.176497
2014-01-08,19.243214,19.484285,19.238930,19.409286,17.038374,258529600,1.006333,-0.031300,-0.031300,-0.040552,...,0.095228,0.287045,51.698823,39.768529,46.104119,-86.761755,4.576474,67.889021,0.065380,-74.242338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,70.132500,71.062500,70.092499,71.000000,69.261597,98572000,1.016318,0.014793,0.014793,0.084880,...,0.090424,0.272606,92.827794,80.791247,74.208465,174.332187,143.745379,135.173461,0.106514,-1.060670
2019-12-24,71.172501,71.222504,70.730003,71.067497,69.327446,48478800,1.000951,0.013766,0.013766,0.067200,...,0.088408,0.271082,93.494583,81.170444,74.436585,164.714255,147.082405,134.947830,0.113020,-2.797949
2019-12-26,71.205002,72.495003,71.175003,72.477501,70.702927,93121200,1.019840,0.036355,0.036355,0.096939,...,0.089774,0.263569,98.668065,87.286880,78.678994,183.170102,169.021135,143.449617,0.123227,-0.279247


In [64]:
def get_top_n_tickers(year, month, n):
    #print("Getting top n tickers for year: " + str(year) + " month: " + str(month))
    results = []
    for ticker in get_current_predictions():
        df = pd.read_csv(PREDICTION_DIR + ticker + '_predictions.csv', index_col='Date', parse_dates=True)
        mask = (df.index.year == year) & (df.index.month == month)
        df = df.loc[mask]
        
        results.append((ticker, df['Returns Predictions'][0]))
    
    #print(results)
    results.sort(key=lambda x: x[1], reverse=True)
    tickers = [i[0] for i in results[:n]]
    pred_vector = [i[1] for i in results[:n]]
    return tickers, pred_vector


def get_top_n_tickers_combined(start_year, start_month, end_year, end_month, n):
    output = []
    pred_vectors = []
    curr_year, curr_month = start_year, start_month
    while not (curr_year > end_year or (curr_year == end_year and curr_month > end_month)):
        tickers, pred_vector = get_top_n_tickers(curr_year, curr_month, n)
        output.append(tickers)
        pred_vectors.append(pred_vector)
        
        if curr_month == 12:
            curr_month = 1
            curr_year += 1
        else:
            curr_month += 1
    return output, pred_vectors


def generate_close_data(tickers, month, year):
    df = pd.DataFrame()
    for ticker in tickers:
        data = pd.read_csv(DATA_DIR + ticker + '.csv',
                           index_col='Date', parse_dates=True)
        data = data.loc[data.index.month == month]
        data = data.loc[data.index.year == year]
        df[ticker] = data['Close']
        df.index = data.index
    return df


def generate_all_close_data(tickers, start_year, start_month, end_year, end_month):
    output = pd.DataFrame()
    curr_year, curr_month = start_year, start_month
    curr_idx = 0
    while not (curr_year > end_year or (curr_year == end_year and curr_month > end_month)):
        data = generate_close_data(tickers[curr_idx], curr_month, curr_year)
        output = pd.concat([output, data], axis=0, join='outer')
        output = output.reset_index(drop=True)
        if curr_month == 12:
            curr_month = 1
            curr_year += 1
        else:
            curr_month += 1
        curr_idx += 1
        
    output.index = sample_index.index
    return output
        
        
    


In [66]:
tickers, pred_vectors = get_top_n_tickers_combined(2014, 1, 2019, 12, 5)

In [68]:
df = generate_all_close_data(tickers, 2014, 1, 2019, 12)

In [78]:
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# Optimize for maximal Sharpe ratio: Mean Variance Optimization
ef = EfficientFrontier(mu, S, weight_bounds=(0, 0.1))
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
#ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)


C:\Users\yinki\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pypfopt\risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(


OrderedDict([('TTWO', 0.00709), ('SWKS', 0.0), ('WELL', 0.00075), ('REGN', 0.0), ('VTR', 0.0), ('ZBRA', 0.00212), ('VTRS', 0.00579), ('TRMB', 0.00034), ('PXD', 0.0), ('VLO', 0.0), ('DGX', 0.0289), ('TSN', 0.03262), ('BK', 0.00743), ('URI', 0.0), ('RMD', 0.0), ('TRV', 0.04571), ('PEG', 0.0851), ('SPGI', 0.03776), ('IPG', 0.0), ('CI', 0.0), ('LUV', 0.00307), ('SYK', 0.01413), ('VRTX', 0.00128), ('WMB', 0.0), ('TYL', 0.0), ('STLD', 0.00062), ('UHS', 0.01142), ('VRSN', 0.00141), ('COO', 0.01184), ('ROST', 0.01058), ('HD', 0.03343), ('TJX', 0.0), ('SEE', 0.0), ('CLX', 0.00355), ('PAYX', 0.0), ('TGT', 0.00072), ('WHR', 0.0), ('TSCO', 0.0), ('TXT', 0.01024), ('WMT', 0.0), ('PHM', 0.00726), ('RCL', 0.0), ('WBA', 0.0099), ('SCHW', 0.02493), ('BA', 0.01157), ('O', 0.0), ('KR', 0.01158), ('VMC', 0.01348), ('EL', 0.00905), ('ROL', 0.03492), ('YUM', 0.00195), ('SLB', 0.04548), ('ZION', 0.0), ('WRB', 0.0), ('SYY', 0.0), ('SJM', 0.0), ('WEC', 0.0), ('UDR', 0.01293), ('PNW', 0.04085), ('PWR', 0.00849)

(0.5131477127878291, 0.020398244741049662, 24.17598764247656)

In [77]:
# Hierarchical Risk Parity
from pypfopt import HRPOpt
hrp = HRPOpt(df)

hrp.optimize()
hrp_weights = hrp.clean_weights()
print(hrp_weights)
hrp.portfolio_performance(verbose=True)




ValueError: The condensed distance matrix must contain only finite values.